In [1]:
#Import dependencies
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import sqlalchemy.ext 
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session

THE ORIGINAL DATA

In [2]:
#Read data and create dataframe
#NOTE: file too large to open in excel

realestate_df = pd.read_csv(r"C:\Desktop\Analysis Projects\Project_4\Resources\realtor-data.zip.csv.zip")
realestate_df


,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2226377,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354.0,3600.0,2022-03-25
2226378,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354.0,1616.0,2022-03-25
2226379,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354.0,3200.0,2022-03-24
2226380,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354.0,933.0,2022-03-24


In [3]:
# Display the data types of the columns
column_dtypes = realestate_df.dtypes
print("Data types of columns:")
print(column_dtypes)

Data types of columns:
brokered_by       float64
status             object
price             float64
bed               float64
bath              float64
acre_lot          float64
street            float64
city               object
state              object
zip_code          float64
house_size        float64
prev_sold_date     object
dtype: object


In [4]:
#Checking to see if there are duplicates in the dataset
# Count duplicate rows based on all columns
num_duplicate_rows = realestate_df.duplicated().sum()

# Display the number of duplicate rows
print(f"Number of Duplicate Rows: {num_duplicate_rows}")

Number of Duplicate Rows: 0


In [5]:
#saving the original file as csv 
# Define the file path for the CSV output
output_file_path = 'realestate_df.csv'

# Save the filtered DataFrame to a CSV file
realestate_df.to_csv(output_file_path, index=False)

In [6]:
#Previewing the dataframe from the original file
realestate_df

,brokered_by,status,price,bed,bath,acre_lot,street,city,state,zip_code,house_size,prev_sold_date
0,103378.0,for_sale,105000.0,3.0,2.0,0.12,1962661.0,Adjuntas,Puerto Rico,601.0,920.0,NaN
1,52707.0,for_sale,80000.0,4.0,2.0,0.08,1902874.0,Adjuntas,Puerto Rico,601.0,1527.0,NaN
2,103379.0,for_sale,67000.0,2.0,1.0,0.15,1404990.0,Juana Diaz,Puerto Rico,795.0,748.0,NaN
3,31239.0,for_sale,145000.0,4.0,2.0,0.10,1947675.0,Ponce,Puerto Rico,731.0,1800.0,NaN
4,34632.0,for_sale,65000.0,6.0,2.0,0.05,331151.0,Mayaguez,Puerto Rico,680.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2226377,23009.0,sold,359900.0,4.0,2.0,0.33,353094.0,Richland,Washington,99354.0,3600.0,2022-03-25
2226378,18208.0,sold,350000.0,3.0,2.0,0.10,1062149.0,Richland,Washington,99354.0,1616.0,2022-03-25
2226379,76856.0,sold,440000.0,6.0,3.0,0.50,405677.0,Richland,Washington,99354.0,3200.0,2022-03-24
2226380,53618.0,sold,179900.0,2.0,1.0,0.09,761379.0,Richland,Washington,99354.0,933.0,2022-03-24


REVIEWING AND PREPARING THE DATA TO BE FILTERED 

In [9]:
# Counts by status
original_status_counts = realestate_df['status'].value_counts()

print("Original Status counts:")
print(original_status_counts )

Original Status counts:
for_sale          1389306
sold               812009
ready_to_build      25067
Name: status, dtype: int64


In [8]:
# Counts by states
state_counts = realestate_df['state'].value_counts()

print("State counts:")
print(state_counts)

State counts:
Florida                 249432
California              227215
Texas                   208335
New York                103159
North Carolina           85745
Illinois                 85280
Georgia                  80977
Pennsylvania             78373
Arizona                  72825
Virginia                 68763
Washington               62461
Ohio                     59207
New Jersey               48199
Maryland                 46052
Missouri                 45145
Minnesota                43412
Michigan                 42429
Wisconsin                42390
South Carolina           42367
Tennessee                40964
Massachusetts            38041
Oklahoma                 37140
Alabama                  34053
Colorado                 32293
Oregon                   32163
Kentucky                 26316
Louisiana                25815
Arkansas                 23045
Iowa                     23033
New Mexico               21074
Indiana                  18840
Idaho                    

In [ ]:
# List of unique states
unique_states = realestate_df['state'].unique()

print("Unique states:")
print(unique_states)

In [ ]:
missing_value_counts =realestate_df.isna().sum()
print("Missing value counts per column:")
print(missing_value_counts)

In [ ]:
#Drop the following columns" brokered_by, street
# Delete columns "brokered_by" and "street"
del realestate_df['brokered_by']
del realestate_df['street']

# Display the updated DataFrame
print(realestate_df)


In [ ]:
# Specify columns to check for null values
columns_to_check = ['price', 'bed', 'bath', 'status', 'acre_lot','city','state','zip_code','house_size']

# Find and drop rows with null values in specified columns
realestate_df.dropna(subset=columns_to_check, inplace=True)

# Display the updated DataFrame
print(realestate_df)

Filtering dataset to exclude non-contiguous states and null values except for prev_sold_date.

In [ ]:
# List of states to exclude (rows with these states will be removed)
states_to_exclude = ['Guam', 'Puerto Rico', 'Virgin Islands','New Brunswick','Hawaii','Alaska']

# Filter rows where state is NOT in the list of states to exclude
filtered_df = realestate_df[~realestate_df['state'].isin(states_to_exclude)]

# Display the filtered DataFrame
print(filtered_df)

In [ ]:
# List of unique states
unique_states = filtered_df['state'].unique()

print("Unique states:")
print(unique_states)

CREATION OF A NEW DATAFRAME FOR DATA ANALYSIS

In [ ]:
#Creation of a filtered data frame

filtered_df.dtypes

In [ ]:
# Counts by states
state_counts = filtered_df['state'].value_counts()

print("State counts:")
print(state_counts)

In [ ]:
# Counts by status
status_counts = realestate_df['status'].value_counts()

print("Status counts:")
print(status_counts )

Saving to export new filtered dataframe as a csv file to be used in a database

In [ ]:

# Define the file path for the CSV output
output_file_path = 'filtered_df.csv'

# Save the filtered DataFrame to a CSV file
filtered_df.to_csv(output_file_path, index=False)

Using Sqlachemy to create a sqlite database connection

In [ ]:
#Read data and create dataframe

filtered_df = pd.read_csv(r"C:\Desktop\Analysis Projects\Project_4\Resources\filtered_df.csv")
#filtered_df

# Create a SQLite database engine
engine = create_engine('sqlite:///filtered_df.sqlite')

# reflect an existing database into a new model
Base=automap_base()

# reflect the tables
Base.prepare(autoload_with=engine)

# Write DataFrame to SQLite database
filtered_df.to_sql('realtor_filtered', con=engine, if_exists='replace', index=False)

# Confirm the data has been written by querying the database
query = "SELECT * FROM realtor_filtered LIMIT 5;"  # Example query
result = engine.execute(query)

Reading the US_GeoCode csv file, renamed column header, and export it as a csv to be used in database

In [ ]:
#Read and create a dataframe for the US_GeoCodes file
# Specify the file path using raw string or escape backslashes
file_path =(r"C:\Desktop\Analysis Projects\Project_4\Resources\stateregion.csv")

# Read data from CSV file into a DataFrame
stateregion_df = pd.read_csv(file_path)

# Display the DataFrame (optional)
print(stateregion_df)



In [ ]:
#DEMONSTRATING PLOTTING THE STATES COUNT IN A BAR CHART ORDERED BY STATE COUNTS

state_counts = filtered_df['state'].value_counts()

# Plotting the state counts as a bar chart using Matplotlib
plt.figure(figsize=(15, 10))  # Set the figure size (width, height) in inches

state_counts.plot(kind='bar', color='skyblue')  # Plotting a bar chart
plt.title('Count of Properties by State')  # Adding a title to the plot
plt.xlabel('State')  # Adding label to x-axis
plt.ylabel('Count')  # Adding label to y-axis
plt.xticks(rotation=90)  # Rotating x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Adding gridlines to y-axis

plt.show()  # Display the plot

MERGING THE REGION AND DIVISION COLUMNS TO THE FILTERED TABLE TO CREATE CHARTS

In [ ]:
# Merge 'realestate_filtered' with 'stateregion' on 'state' column
# Merge based on different column names: 'state' in realestate_filtered and 'name' in stateregion
#merged_df = pd.merge(filtered_df, stateregion_df[['region','division']], left_on='state', right_on='name', how='left')

stateregion_subset = stateregion_df[['name', 'region', 'division']]
merged_df = pd.merge(filtered_df, stateregion_subset, left_on='state', right_on='name', how='left')
merged_df.drop('name', axis=1, inplace=True)

# Display the merged dataframe
print(merged_df)



In [ ]:
region_counts = merged_df['region'].value_counts()

# Plotting the state counts as a bar chart using Matplotlib
plt.figure(figsize=(15, 10))  # Set the figure size (width, height) in inches

region_counts.plot(kind='bar', color='skyblue')  # Plotting a bar chart
plt.title('Count of Properties by Region')  # Adding a title to the plot
plt.xlabel('Region')  # Adding label to x-axis
plt.ylabel('Count')  # Adding label to y-axis
plt.xticks(rotation=45)  # Rotating x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Adding gridlines to y-axis

plt.show()  # Display the plot

In [ ]:
division_counts = merged_df['division'].value_counts()

# Plotting the division counts as a bar chart using Matplotlib
plt.figure(figsize=(15, 10))  # Set the figure size (width, height) in inches

division_counts.plot(kind='bar', color='skyblue')  # Plotting a bar chart
plt.title('Count of Properties by Division')  # Adding a title to the plot
plt.xlabel('Division')  # Adding label to x-axis
plt.ylabel('Count')  # Adding label to y-axis
plt.xticks(rotation=45)  # Rotating x-axis labels for better readability
plt.grid(axis='y', linestyle='--', alpha=0.7)  # Adding gridlines to y-axis

plt.show()  # Display the plot

In [ ]:

# Ploting the Average house_size by region and status

# Calculate average house_size by region and status
average_house_size = merged_df.groupby(['region', 'status'])['house_size'].mean().reset_index()

# Pivot the data to prepare for plotting
pivot_table = average_house_size.pivot(index='region', columns='status', values='house_size')

# Plotting the average house_size by region and status
plt.figure(figsize=(20, 10))
pivot_table.plot(kind='bar', color=['blue', 'orange'], alpha=0.8)

# Add labels and title
plt.title('Average House Size by Region and Status')
plt.xlabel('Region')
plt.ylabel('Average House Size(Sqft)')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability

# Show legend at the bottom
plt.legend(title='Status', bbox_to_anchor=(-0.5, -0.60), loc='lower left', ncol=2)

# Show plot
plt.grid(axis='y')  # Show gridlines on y-axis
plt.tight_layout()  # Adjust layout to prevent overlapping labels
plt.show()